<a href="https://colab.research.google.com/github/simpykumarimandal/extract_text/blob/main/Untitled27.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import cv2
import numpy as np

# Load Image
image = cv2.imread("prescription.jpg")

# Convert to Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Gaussian Blur (to remove noise)
blurred = cv2.GaussianBlur(gray, (5,5), 0)

# Apply Adaptive Thresholding (to enhance contrast)
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Morphological Operations (Dilation to enhance text)
kernel = np.ones((2,2), np.uint8)
dilated = cv2.dilate(thresh, kernel, iterations=1)

# Sharpening Filter
sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
sharpened = cv2.filter2D(dilated, -1, sharpen_kernel)

# Save & Show Result
cv2.imwrite("enhanced_prescription.jpg", sharpened)
cv2.imshow("Enhanced Image", sharpened)
cv2.waitKey(0)
cv2.destroyAllWindows()


DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow


In [12]:
import cv2
import numpy as np

# Load Image
image = cv2.imread("prescription.jpg")

# Convert to Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Contrast Enhancement (Histogram Equalization)
equalized = cv2.equalizeHist(gray)

# Apply Gaussian Blur (to reduce noise)
blurred = cv2.GaussianBlur(equalized, (5,5), 0)

# Adaptive Thresholding for Better Text Clarity
thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Increase Text Thickness (Dilation)
kernel = np.ones((3,3), np.uint8)  # Increase kernel size to make text bolder
bold_text = cv2.dilate(thresh, kernel, iterations=2)

# Upscale Image to Increase Font Size
upscaled = cv2.resize(bold_text, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

# Save & Show Result
cv2.imwrite("enhanced_prescription.jpg", upscaled)
cv2.imshow("Enhanced Image", upscaled)
cv2.waitKey(0)
cv2.destroyAllWindows()


DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow


In [4]:
pip install pytesseract

ERROR: unknown command "!"


In [5]:
pip install pytesseract


In [8]:
!apt update && apt install -y tesseract-ocr
!pip install pytesseract


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,762 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,677 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692

In [9]:
!tesseract --version


tesseract 4.1.1
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8


In [10]:
import pytesseract

pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"



In [11]:
import cv2
import pytesseract
import re
import json

# Load Image
image = cv2.imread("prescription.jpg")

# Convert to Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Adaptive Thresholding (Enhance contrast)
processed_image = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Extract Text using Tesseract OCR
text = pytesseract.image_to_string(processed_image)
print("Extracted Text:\n", text)

# Function to Extract Key Information
def extract_info(text):
    prescription_data = {}

    # Extract Patient Name
    patient_match = re.search(r'Patient[:\-]?\s*([A-Za-z\s]+)', text)
    prescription_data["Patient Name"] = patient_match.group(1).strip() if patient_match else "Unknown"

    # Extract Doctor Name
    doctor_match = re.search(r'Dr\.?\s*([A-Za-z\s]+)', text)
    prescription_data["Doctor Name"] = doctor_match.group(1).strip() if doctor_match else "Unknown"

    # Extract Medicines
    medicine_pattern = r"([A-Za-z]+)\s*(\d+mg)?\s*(OD|BD|TDS|QID|PRN|HS|SOS)?"
    medicines = re.findall(medicine_pattern, text)

    prescription_data["Medicines"] = []
    for med in medicines:
        name, dosage, frequency = med
        if name.strip():  # Avoid empty matches
            prescription_data["Medicines"].append({
                "Name": name.strip(),
                "Dosage": dosage.strip() if dosage else "Unknown",
                "Frequency": frequency.strip() if frequency else "Unknown"
            })

    return prescription_data

# Extract & Structure Data
structured_data = extract_info(text)

# Convert to JSON
json_output = json.dumps(structured_data, indent=4)
print("\nStructured Prescription Data:\n", json_output)

# Save as JSON File
with open("prescription.json", "w") as json_file:
    json_file.write(json_output)


Extracted Text:
  


Structured Prescription Data:
 {
    "Patient Name": "Unknown",
    "Doctor Name": "Unknown",
    "Medicines": []
}


In [15]:
# Apply Gaussian Blur to reduce noise
blurred = cv2.GaussianBlur(gray, (5,5), 0)

# Use Otsu's thresholding for better binarization
_, otsu_thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Sharpen the image to enhance text clarity
sharpen_kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
sharpened = cv2.filter2D(otsu_thresh, -1, sharpen_kernel)

# Apply OCR again
text_refined = pytesseract.image_to_string(image.fromarray(sharpened), lang="eng")

# Output the extracted text
text_refined


AttributeError: 'numpy.ndarray' object has no attribute 'fromarray'

In [16]:
# Further enhance the image for handwritten text extraction

# Invert the image to make text stand out
inverted = cv2.bitwise_not(sharpened)

# Apply dilation to make text bolder
dilated = cv2.dilate(inverted, np.ones((2,2), np.uint8), iterations=1)

# Run OCR again with custom configuration for handwritten text
custom_config = "--psm 6"  # Mode for sparse text
handwritten_text = pytesseract.image_to_string(image.fromarray(dilated), config=custom_config, lang="eng")

# Output the extracted handwritten text
handwritten_text


AttributeError: 'numpy.ndarray' object has no attribute 'fromarray'

In [7]:
import cv2
import pytesseract

# Load image
image = cv2.imread("prescription.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply OCR
text = pytesseract.image_to_string(gray)
print(text)


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.

In [17]:
import cv2
import numpy as np
import pytesseract
from PIL import Image

# Load Image
image = cv2.imread("prescription.jpg")

# Convert to Grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply Adaptive Thresholding
thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)

# Apply Dilation to Make Text Bolder
kernel = np.ones((2,2), np.uint8)
dilated = cv2.dilate(thresh, kernel, iterations=1)

# Save Processed Image for Debugging
cv2.imwrite("processed_prescription.jpg", dilated)

# Apply OCR
text = pytesseract.image_to_string(Image.fromarray(dilated), config="--psm 6", lang="eng")

print(text)  # Check the extracted text


G : fe 5a :
' ENTRE
' - a
“CITY SKIN CARE C scones
oa a BANK mya F2 (ad -
- C re infront of HOFCE AN eo eg en ET
OL Seraitinata, Dhonbs ane A
DeS. ke Hel Soe ala fli se Be in YT BSE TTS
WBE ans ed Re ir
. (Sein VO Lepresy} Patna. So urate Tea = to f.30 p.m
eer ibowe Eo Flo aemncan 12.30.02 Tv
other aer BOM a pemnean #253pan 2 LN
Contael Deraaletog O° we, Bed tg 130 Bm. Las, .
i La 2 [ese . a
Tenet! eo i a
: Toney Sane Date sienvrenene .
SoS ESE _
a P Pelive e
wo ae Ye wet ee!
i « re bess :
CO ny {Pasitbabee = 4 Pe Ee :
*S. ‘ . Ebidaeqe te, hee Gp berete rs so
re on Pe Pee Ona aa Pe
| feet SES INE Pe pes . ‘ oR i
ye set Nak Ma A Ty
eat : Bay rege BT le eS
ne me =ber, pee ye Lae tot i ¢
wean eeams reich he Watters SO 2).
1b oe set a eo Sp “AY,
vege e headin ce PF, ne fk f 4,
ce MN.
: Pf f gate - we 2 . ” ae oe
JP lyre “ . - g : if |!
’ Noe] tae TU dene magiretine oT
ey ae &, . ee Vee dene ee
a { SS Trip cA Sefever: foe Had . Be Eye ors * wet r
. vy) Shales Hop tm Seeda dade! Boys Ve ogta

In [18]:
custom_config = r'--oem 3 --psm 6'
handwritten_text = pytesseract.image_to_string(Image.fromarray(dilated), config=custom_config, lang="eng")
print(handwritten_text)


G : fe 5a :
' ENTRE
' - a
“CITY SKIN CARE C scones
oa a BANK mya F2 (ad -
- C re infront of HOFCE AN eo eg en ET
OL Seraitinata, Dhonbs ane A
DeS. ke Hel Soe ala fli se Be in YT BSE TTS
WBE ans ed Re ir
. (Sein VO Lepresy} Patna. So urate Tea = to f.30 p.m
eer ibowe Eo Flo aemncan 12.30.02 Tv
other aer BOM a pemnean #253pan 2 LN
Contael Deraaletog O° we, Bed tg 130 Bm. Las, .
i La 2 [ese . a
Tenet! eo i a
: Toney Sane Date sienvrenene .
SoS ESE _
a P Pelive e
wo ae Ye wet ee!
i « re bess :
CO ny {Pasitbabee = 4 Pe Ee :
*S. ‘ . Ebidaeqe te, hee Gp berete rs so
re on Pe Pee Ona aa Pe
| feet SES INE Pe pes . ‘ oR i
ye set Nak Ma A Ty
eat : Bay rege BT le eS
ne me =ber, pee ye Lae tot i ¢
wean eeams reich he Watters SO 2).
1b oe set a eo Sp “AY,
vege e headin ce PF, ne fk f 4,
ce MN.
: Pf f gate - we 2 . ” ae oe
JP lyre “ . - g : if |!
’ Noe] tae TU dene magiretine oT
ey ae &, . ee Vee dene ee
a { SS Trip cA Sefever: foe Had . Be Eye ors * wet r
. vy) Shales Hop tm Seeda dade! Boys Ve ogta

In [19]:
import re

# Sample extracted text
raw_text = """
   Ssfroza 20
   Lacsoht moisture
   Milly Antidnruff Shampee
"""

# Common replacements for OCR errors
corrections = {
    "Ssfroza": "Safroza",
    "Lacsoht": "Lacsoft",
    "Milly": "Mildy",
    "Shampee": "Shampoo",
    "Antidnruff": "Anti-Dandruff"
}

# Apply corrections
for wrong, right in corrections.items():
    raw_text = raw_text.replace(wrong, right)

# Remove unwanted characters
clean_text = re.sub(r"[^a-zA-Z0-9\s]", "", raw_text)

print(clean_text)



   Safroza 20
   Lacsoft moisture
   Mildy AntiDandruff Shampoo



In [21]:
pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [22]:
import easyocr

reader = easyocr.Reader(['en'])  # Load English OCR model
result = reader.readtext("prescription.jpg")

# Print extracted text
for detection in result:
    print(detection[1])  # Extracted text part



Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteCity SKIN CARE CENTRE_
zio (eo #0 #e8
In front of HDFC =
@w @ @ Io &)
Saraidhela,
1271
D~ $ K Mandal
4i4 @1147 % F414;
Ptlttenr
MB.B.S. (Ranchil
(4,a5, 134, 3 7,
98
MD. (Skin VD Lcprosy) Patna.
44 7 @n
Life Member IADVL
To Fio Forf #f375 #av
Timb
Consultant Dermatologist
1.30 P.m:
T
Afternaon - 12.30 pm to
7.00 pm: t0 &.00 pm:
Evedang- 12.00 pm: t0 1 30 Pm;
Sunday
Jei
Fumnol
6'Jz
5 Di5
Date
Gintive
Jex
Aetbl
Du~i (imjL
Pco $
Tnun
Ic+
an'
61
kvn
Ebiane
f~
L J~
Synesvm"
Ih L'
'
An k
8vccua
Iu~ -
6 6JL
p 64
ALia
Aclan
J~
pnleld
9i (
Fn
Ac44
00
Saferel
Ru =
hn
filne
Iep
Fty ~ k 44
ckxplr
Fap
I~1
Jevuu Jax
Fy
eln 3
Slul' _
Any
b~rbub
peamuar
Fe
NeLL
Knl`
La
cA~J
b+~
garetl
Mi 'm (
S~
51 -
Bw
hnw
Dent]
Tc
T 4T JquiT t3Us?
34ifed WHI4 Torr
9 Flhno % Jv Iri 47 0 &i &4 #
ciasst
M14 W Z4T Tr Tr gra
15 f7 # %ect 1/2 4 cpf 3RZ
97 alis d
daly
15 Rv & a 5er % wpim
dlo
' 9835989591,7488511706
BANK
Dhanbad
JW
(a)
N(
"a4
o2/0